# Import libraries

In [93]:
!pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2022-02-10 01:20:34 +01:00)


In [53]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageFile
from skimage import io
from skimage.transform import rescale
from skimage.transform import resize


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier # Pour instancier et entraîner un modèle Random Forest
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # Pour les performances de classification

In [54]:
import os
import glob
import json
from tqdm import tqdm
import pandas as pd


from PIL import Image
import imagesize


import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')
from IPython.display import display

In [55]:
#Directory path in my com
root_dir ='C:/Users/ANTOINE/Project_Mushroom/' 
raw_data_dir = root_dir + 'raw_data/' #raw images folder
images_dir = 'row_data/images'
json_dir = raw_data_dir + 'json'
ML_dir = root_dir + 'ML'

# Rebuild database

In [56]:
rebuild_database = True

In [47]:
if rebuild_database:
  # list all year directories
  year_dirs = [f.path for f in os.scandir(images_dir) if f.is_dir()]
  # loop to get a csv per year 
  for year_dir in year_dirs:
    year = year_dir[-4:]
    csv_path = root_dir + year + '.csv'
    img_dict = {}
    for path, subdirs, files in tqdm(os.walk(year_dir)):
          for name in files:
              file_path = os.path.join(path, name) # get path
              resolution= (imagesize.get(file_path)) # get res
              img_dict[name]= [file_path,resolution] #append to dict
          # create df and save to not redo each time
          img_lst_df = pd.DataFrame.from_dict(img_dict, orient='index',columns=["file_path","resolution"])
          img_lst_df['file_year'] = year
          img_lst_df.to_csv(csv_path)

42402it [4:04:03,  2.90it/s] 
1it [00:00,  1.38it/s]


In [98]:
df = pd.read_csv(root_dir + 'ages.csv')
df

,Unnamed: 0,file_path,resolution,file_year
0,1208.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(243, 320)",ages
1,1604.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 164)",ages
2,1608.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 286)",ages
3,29.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages
4,680.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages
...,...,...,...,...
214851,625253.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 229)",ages
214852,630748.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 276)",ages
214853,662083.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 299)",ages
214854,678141.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(214, 320)",ages


time: 328 ms (started: 2022-02-10 01:26:11 +01:00)


In [94]:
if rebuild_database:
  # laod and concat csv per year 
  csv_files = [root_dir + str(year) + '.csv' for year in range(2006,2017) if os.path.exists(root_dir + str(year) + '.csv')]
  dfs_ = []
  for filename in csv_files:
      df = pd.read_csv(filename, index_col=None, header=0)
      dfs_.append(df)
  img_lst_df = pd.concat(dfs_, axis=0, ignore_index=True)
  # extract iamge_id and format 
  img_lst_df.rename(columns={img_lst_df.columns[0]: "file_name"}, inplace=True)
  img_lst_df[['image_id', 'format']] = img_lst_df['file_name'].str.split('.', 1, expand=True)
  # display(img_lst_df.head())

ValueError: No objects to concatenate

time: 15 ms (started: 2022-02-10 01:20:40 +01:00)


In [103]:
img_lst_df = pd.read_csv(root_dir + 'ages.csv')
img_lst_df.rename(columns={img_lst_df.columns[0]: "file_name"}, inplace=True)
img_lst_df[['image_id', 'format']] = img_lst_df['file_name'].str.split('.', 1, expand=True)
img_lst_df

,file_name,file_path,resolution,file_year,image_id,format
0,1208.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(243, 320)",ages,1208,jpg
1,1604.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 164)",ages,1604,jpg
2,1608.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 286)",ages,1608,jpg
3,29.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages,29,jpg
4,680.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages,680,jpg
...,...,...,...,...,...,...
214851,625253.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 229)",ages,625253,jpg
214852,630748.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 276)",ages,630748,jpg
214853,662083.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 299)",ages,662083,jpg
214854,678141.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(214, 320)",ages,678141,jpg


time: 1.02 s (started: 2022-02-10 02:00:14 +01:00)


# Json

In [104]:
if rebuild_database:
  json_pattern = os.path.join(json_dir, '*.json')
  file_list = ['C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_0.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_1.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_2.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_3.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_4.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_5.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_6.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_7.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_8.json',
               'C:/Users/ANTOINE/Project_Mushroom/row_data\json\observations_9.json']
  print(json_pattern)

  json_dfs = []
  for file in file_list:
      with open(file) as f:
          json_data = pd.json_normalize(json.loads(f.read()))
          json_data['json_file'] = file.rsplit("/", 1)[-1]
      json_dfs.append(json_data)
  json_df = pd.concat(json_dfs)

C:/Users/ANTOINE/Project_Mushroom/raw_data/json\*.json
time: 21.6 s (started: 2022-02-10 02:00:20 +01:00)


In [110]:
print(json_df)

       observation               label  image_id  \
0                1  Xylaria polymorpha         1   
1                2   Xylaria magnoliae         2   
2                3   Xylaria hypoxylon         3   
3                4   Xylaria hypoxylon         4   
4                5        Xeromphalina         5   
...            ...                 ...       ...   
68563       256407               Fungi    673194   
68564       256407               Fungi    673195   
68565       256408               Fungi    673196   
68566       256408               Fungi    673197   
68567       256408               Fungi    673198   

                                           image_url  user  \
0           http://mushroomobserver.org/images/320/1     1   
1           http://mushroomobserver.org/images/320/2     1   
2           http://mushroomobserver.org/images/320/3     1   
3           http://mushroomobserver.org/images/320/4     1   
4           http://mushroomobserver.org/images/320/5     1   
...

In [105]:
 # create df and save to not redo each time 
if rebuild_database:
  json_df.to_csv(path_or_buf=root_dir + 'json_data.csv',index=False)

time: 9.17 s (started: 2022-02-10 02:00:42 +01:00)


In [111]:
# load csv 
if rebuild_database:
  json_lst_df = pd.read_csv(root_dir + 'json_data.csv',low_memory=False)
  json_lst_df

time: 3.86 s (started: 2022-02-10 02:02:25 +01:00)


In [115]:
## join to get only file present in image dir and json files 
if rebuild_database:
  json_lst_df['image_id'] = json_lst_df['image_id'].astype(str)
  df = img_lst_df.merge(json_lst_df, how='inner', on='image_id')
  df.head()

time: 2.08 s (started: 2022-02-10 02:06:01 +01:00)


In [116]:
if rebuild_database:
  # save final df for EDA
  df.to_csv(path_or_buf=root_dir + 'image_and_json_data.csv',index=False)

time: 3 s (started: 2022-02-10 02:06:07 +01:00)


In [117]:
# load final database
df =  pd.read_csv(root_dibr + 'image_and_json_data.csv')

time: 1.36 s (started: 2022-02-10 02:06:13 +01:00)


In [118]:
df



,file_name,file_path,resolution,file_year,image_id,format,observation,label,image_url,user,...,gbif_info.canonicalName,gbif_info.phylumKey,gbif_info.class,gbif_info.synonym,gbif_info.scientificName,gbif_info.genus,gbif_info.order,gbif_info.note,gbif_info,json_file
0,1208.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(243, 320)",ages,1208,jpg,1007,Agaricales,http://mushroomobserver.org/images/320/1208,25,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
1,1604.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 164)",ages,1604,jpg,1535,Agaricales,http://mushroomobserver.org/images/320/1604,43,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
2,1608.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 286)",ages,1608,jpg,1537,Agaricales,http://mushroomobserver.org/images/320/1608,43,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
3,29.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages,29,jpg,23,Agaricales,http://mushroomobserver.org/images/320/29,1,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
4,680.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages,680,jpg,536,Agaricus,http://mushroomobserver.org/images/320/680,1,...,Agaricus,34.0,Agaricomycetes,False,Agaricus,Agaricus,Agaricales,NaN,NaN,row_data\json\observations_0.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208769,623678.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 253)",ages,623678,jpg,240888,Xylocoremium flabelliforme,http://mushroomobserver.org/images/320/623678,6466,...,Xylocoremium flabelliforme,95.0,Sordariomycetes,False,Xylocoremium flabelliforme (Schwein.) J.D. Rog...,Xylocoremium,Xylariales,NaN,NaN,row_data\json\observations_9.json
208770,625253.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 229)",ages,625253,jpg,241421,Xylocoremium flabelliforme,http://mushroomobserver.org/images/320/625253,4186,...,Xylocoremium flabelliforme,95.0,Sordariomycetes,False,Xylocoremium flabelliforme (Schwein.) J.D. Rog...,Xylocoremium,Xylariales,NaN,NaN,row_data\json\observations_9.json
208771,630748.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 276)",ages,630748,jpg,242977,Xylocoremium flabelliforme,http://mushroomobserver.org/images/320/630748,5796,...,Xylocoremium flabelliforme,95.0,Sordariomycetes,False,Xylocoremium flabelliforme (Schwein.) J.D. Rog...,Xylocoremium,Xylariales,NaN,NaN,row_data\json\observations_9.json
208772,662083.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 299)",ages,662083,jpg,252700,Xylocoremium flabelliforme,http://mushroomobserver.org/images/320/662083,5796,...,Xylocoremium flabelliforme,95.0,Sordariomycetes,False,Xylocoremium flabelliforme (Schwein.) J.D. Rog...,Xylocoremium,Xylariales,NaN,NaN,row_data\json\observations_9.json


time: 203 ms (started: 2022-02-10 02:06:25 +01:00)
